# Trabajo Practico 1: Bayes Ingenuo

### Estudiantes:
1. Sophia Contreras
2. Yoksan Varela
3. Mauro Viquez

In [1]:
# Librerias usadas en el codigo
import torch
import numpy as np
import torchvision
from PIL import Image
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms

## Funciones Generales

In [2]:
def load_cifar10_dataset(is_train = True):
    """Funcion para cargar CIFAR10 dataset, tomada del notebook BayesianModel_CIFAR_base.ipynb provisto en el curso.

    Args:
        is_train (bool, optional): Especifica si el modelo se esta entrenando. Defaults to True.

    Returns:
        Tensor: Retorna los tensores con sus respectivos labels
    """
    # Define a transformation to convert images to grayscale
    transforms_1 = transforms.Compose([
      transforms.ToTensor(),
      transforms.Grayscale(num_output_channels=1)  # Convert to grayscale
    ])
    cifar_trainset = datasets.CIFAR10(root='./data', train = is_train, download = True, transform = transforms_1)
  

    # Initialize an empty list to store batches
    all_data = []
    train_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size = 64, shuffle=True)
    # Iterate over the train_loader to fetch all batches
    for batch in train_loader:
        images, _ = batch  # Extract images from the batch
        all_data.append(images)

    # Concatenate all batches into a single tensor along the batch dimension
    cifar_trainset_tensor = torch.round(torch.cat(all_data, dim=0) * 255)
    cifar_labels = torch.tensor(cifar_trainset.targets)
    print("cifar_trainset_tensor shape ", cifar_trainset_tensor.shape)
    print("cifar_labels ", cifar_labels.shape)
    return (cifar_trainset_tensor, cifar_labels)

In [17]:
def calcular_probabilidad_priori(labels,categoria):
    N = len(labels)
    cuenta_categoria = np.count_nonzero(labels == categoria)
    print(f"Cuenta para categori={categoria} es {cuenta_categoria}")
    prob_marginal = cuenta_categoria/N
    return torch.tensor(prob_marginal)

## Pregunta 1: Implementacion de la clasificacion multi-clase de imagenes con Byes Ingenuo usando histogramas

Como primer paso, tenemos que cargar el set de datos:

In [4]:
# Cargando el dataset CIFAR10
trainset_tensor,trainset_labels = load_cifar10_dataset()

Files already downloaded and verified
cifar_trainset_tensor shape  torch.Size([50000, 1, 32, 32])
cifar_labels  torch.Size([50000])


Ya con el dataset cargado, se analiza la distribucion de la informacion con respecto a las categorias:

In [27]:
valores_unicos, cuenta = np.unique(trainset_labels,return_counts=True)
print(f"Lista de los valores unicos de categorias: {valores_unicos}")
print(f"Cuentas para cada uno de estos valores: {cuenta}")

Lista de los valores unicos de categorias: [0 1 2 3 4 5 6 7 8 9]
Cuentas para cada uno de estos valores: [5000 5000 5000 5000 5000 5000 5000 5000 5000 5000]


Hay que hacer un sampling dado que los 50000 valores estan igualmente distribuidos en las 10 categorias:

In [31]:
# Primero, creamos un arreglo del tamano de los datos con valores aleatorios, de forma que estos van a ser la forma sampleo ya que serian los indices a mantener
arr_sampleo = np.random.choice(np.arange(len(trainset_labels)), 10000, replace=False)

# Ahora se hace el sampleo tanto de imagenes como las categorias
sample_labels = trainset_labels[arr_sampleo]
sample_images = trainset_tensor[arr_sampleo,:,:,:]

In [32]:
# Ahora se comprueba que el sampling se haya hecho de forma de correcta
print("Sample of images shape ", sample_images.shape)
print("Sample of labels ", sample_labels.shape)

Sample of images shape  torch.Size([10000, 1, 32, 32])
Sample of labels  torch.Size([10000])


In [33]:
# Verificamos que ya haya aleatoriedad en las categorias
valores_unicos_sample, cuenta_sample = np.unique(sample_labels,return_counts=True)
print(f"Lista de los valores unicos de categorias: {valores_unicos_sample}")
print(f"Cuentas para cada uno de estos valores: {cuenta_sample}")

Lista de los valores unicos de categorias: [0 1 2 3 4 5 6 7 8 9]
Cuentas para cada uno de estos valores: [ 991  976 1014  979  961 1050 1014  998 1026  991]


In [37]:
indice_1 = 0
indice_2 = 4
indice_3 = 7
print(f"La probabilidad marginal de {indice_1} es {calcular_probabilidad_priori(sample_labels,indice_1)}")
print(f"La probabilidad marginal de {indice_2} es {calcular_probabilidad_priori(sample_labels,indice_2)}")
print(f"La probabilidad marginal de {indice_3} es {calcular_probabilidad_priori(sample_labels,indice_3)}")

Cuenta para categori=0 es 991
La probabilidad marginal de 0 es 0.09910000115633011
Cuenta para categori=4 es 961
La probabilidad marginal de 4 es 0.09610000252723694
Cuenta para categori=7 es 998
La probabilidad marginal de 7 es 0.0997999981045723
